# FIPS Code + Shipment Data

In [1]:
# Importing required packages
import pandas as pd
import numpy as np
import warnings

# Set default option
pd.set_option("mode.copy_on_write", True)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
# Load dataset
shipment = pd.read_parquet("../../01_Data/02_Processed/04_Shipment_Combined.parquet")
fips_codes = pd.read_csv("../../01_Data/01_Raw/county_fips.csv")

In [3]:
shipment.sample(5)

,BUYER_STATE,BUYER_COUNTY,YEAR,MME
32722,SD,BEADLE,2009,2.242618e+06
3196,CO,DELTA,2013,1.163232e+07
18968,MO,BOONE,2017,4.178440e+07
12206,KS,GRAY,2018,2.247214e+06
42570,WV,WOOD,2018,1.681156e+07


In [4]:
fips_codes.sample(5)

,BUYER_COUNTY,BUYER_STATE,countyfips
2545,BURNET,TX,48053
429,DOOLY,GA,13093
850,MITCHELL,IA,19131
16,COLBERT,AL,1033
2452,GRAINGER,TN,47057


In [5]:
# Perform the merge operation directly since the columns are already in all caps
shipment_fips = pd.merge(
    shipment,
    fips_codes,
    on=["BUYER_COUNTY", "BUYER_STATE"],
    how="left",
)
shipment_fips.sample(5)

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,countyfips
32356,SC,FLORENCE,2007,6.569711e+07,45041.0
36917,TX,NEWTON,2015,2.231951e+06,48351.0
16234,MD,WORCESTER,2013,2.518266e+07,24047.0
33503,TN,BRADLEY,2014,9.183759e+07,47011.0
8473,IA,WRIGHT,2013,3.769308e+06,19197.0


In [6]:
shipment_fips.describe()

,YEAR,MME,countyfips
count,42908.000000,4.290800e+04,41742.000000
mean,2012.500583,3.408449e+07,30442.003833
std,4.031702,1.897021e+08,15005.114640
min,2006.000000,1.816200e+01,1001.000000
25%,2009.000000,2.386080e+06,19015.000000
50%,2013.000000,8.205714e+06,29134.000000
75%,2016.000000,2.577040e+07,45049.000000
max,2019.000000,3.405486e+10,56045.000000


In [7]:
shipment_fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42908 entries, 0 to 42907
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BUYER_STATE   42908 non-null  object 
 1   BUYER_COUNTY  42908 non-null  object 
 2   YEAR          42908 non-null  int32  
 3   MME           42908 non-null  float64
 4   countyfips    41742 non-null  float64
dtypes: float64(2), int32(1), object(2)
memory usage: 1.5+ MB


In [8]:
# Fill NaN values with a placeholder (0) before converting to integers
shipment_fips["countyfips"] = (
    shipment_fips["countyfips"].fillna(0).astype(int).astype(str).str.zfill(5)
)

# Replace the placeholder '00000' back to NaN
shipment_fips["countyfips"] = shipment_fips["countyfips"].replace("00000", np.nan)

# Check the first few rows of the adjusted dataset
shipment_fips.sample(5)

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,countyfips
36594,TX,LLANO,2008,4.700442e+06,48299
27512,OH,ERIE,2019,1.396080e+07,39043
32504,SC,LEE,2015,3.911596e+06,45061
37476,TX,TAYLOR,2019,2.301412e+07,48441
21830,MT,JUDITH BASIN,2008,3.027000e+02,30045


In [9]:
shipment_fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42908 entries, 0 to 42907
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BUYER_STATE   42908 non-null  object 
 1   BUYER_COUNTY  42908 non-null  object 
 2   YEAR          42908 non-null  int32  
 3   MME           42908 non-null  float64
 4   countyfips    41742 non-null  object 
dtypes: float64(1), int32(1), object(3)
memory usage: 1.5+ MB


In [10]:
# Drop the rows where 'BUYER_STATE' is 'AR' from the shipment_fips DataFrame, Montgomery changed name.
# shipment_fips = shipment_fips[shipment_fips["BUYER_COUNTY"] == "MONTGOMERY"]["BUYER_STATE"].unique()

In [11]:
# Checking for NaNs
nan_fips = shipment_fips[shipment_fips["countyfips"].isna()]

# Group by 'BUYER_STATE' and 'BUYER_COUNTY' and count the NaN values
nan_counts = (
    nan_fips.groupby(["BUYER_STATE", "BUYER_COUNTY"])
    .size()
    .reset_index(name="NaN_count")
)

# Print the result
nan_counts.head(5)

,BUYER_STATE,BUYER_COUNTY,NaN_count
0,AR,MONTGOMERY,14
1,GU,GUAM,14
2,MP,NORTHERN MARIANA ISLANDS,9
3,MP,SAIPAN,5
4,PR,ADJUNTAS,14


In [12]:
# Dropping rows with NaNs for FIPS codes
shipment_fips = shipment_fips.dropna(subset=["countyfips"])

In [13]:
# confirming that there are no more NaNs
shipment_fips.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
MME             0
countyfips      0
dtype: int64

# Merged shipment mapped with Population

In [14]:
# Load dataset
population = pd.read_parquet("../../01_Data/02_Processed/01_Population.parquet")
state_codes = pd.read_csv("../../01_Data/01_Raw/us_states-ab.csv")

In [15]:
population.sample(5)

,State,State_Code,County,County_Code,Year,Population
35556,Texas,48,WHEELER,48483,2004,4934
23618,New York,36,FULTON,36035,2013,54353
6898,Idaho,16,BONNER,16017,2011,40795
35802,Utah,49,DUCHESNE,49013,2003,15002
10081,Iowa,19,CEDAR,19031,2009,18458


In [16]:
state_codes.sample(5)

,state,abbrev,code
47,Washington,Wash.,WA
23,Minnesota,Minn.,MN
43,Texas,Tex.,TX
13,Illinois,Ill.,IL
8,District of Columbia,D.C.,DC


In [17]:
# Merge the population dataset with the state abbreviations dataset
population_state_code = pd.merge(
    population, state_codes, left_on="State", right_on="state", how="left"
)

# Drop the columns we don't need
population_state_code = population_state_code.drop(
    columns=["state", "abbrev", "State_Code"]
)

# Rename the 'code' column from the state_codes DataFrame to 'State_Code'
population_state_code.rename(columns={"code": "State_Code"}, inplace=True)
population_state_code.head(5)

,State,County,County_Code,Year,Population,State_Code
0,Alabama,AUTAUGA,01001,2003,46800,AL
1,Alabama,AUTAUGA,01001,2004,48366,AL
2,Alabama,AUTAUGA,01001,2005,49676,AL
3,Alabama,AUTAUGA,01001,2006,51328,AL
4,Alabama,AUTAUGA,01001,2007,52405,AL


In [18]:
# checking for NaNs
population_state_code.isna().sum()

State          0
County         0
County_Code    0
Year           0
Population     0
State_Code     0
dtype: int64

In [19]:
# Convert 'year' column to nullable integer if it's in 'shipment_fips'
shipment_fips["YEAR"] = shipment_fips["YEAR"].astype("Int64")

# Ensure county names and state codes are in the same format
shipment_fips["BUYER_COUNTY"] = shipment_fips["BUYER_COUNTY"].str.upper().str.strip()
population_state_code["County"] = (
    population_state_code["County"].str.upper().str.strip()
)

# Convert 'population' column to nullable integer if it's in 'population_state_code'
population_state_code["Population"] = population_state_code["Population"].astype(
    "Int64"
)


# Merge datasets on BUYER_STATE with State_Code, BUYER_COUNTY with County, and YEAR with Year
shipment_with_population = pd.merge(
    shipment_fips,
    population_state_code,
    left_on=["BUYER_STATE", "BUYER_COUNTY", "YEAR"],
    right_on=["State_Code", "County", "Year"],
    how="left",
)

# Check  few rows
shipment_with_population.sample(5)

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,countyfips,State,County,County_Code,Year,Population,State_Code
9277,IL,DEKALB,2011,1.520469e+07,17037,Illinois,DEKALB,17037,2011.0,104466,IL
13455,KY,CHRISTIAN,2010,2.103909e+07,21047,Kentucky,CHRISTIAN,21047,2010.0,74145,KY
8424,IA,WORTH,2006,2.500260e+05,19195,Iowa,WORTH,19195,2006.0,7649,IA
12662,KS,OSAGE,2015,3.137677e+06,20139,Kansas,OSAGE,20139,2015.0,15867,KS
8853,ID,LINCOLN,2018,4.602699e+05,16063,NaN,NaN,NaN,NaN,<NA>,NaN


In [20]:
shipment_with_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41742 entries, 0 to 41741
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BUYER_STATE   41742 non-null  object 
 1   BUYER_COUNTY  41742 non-null  object 
 2   YEAR          41742 non-null  Int64  
 3   MME           41742 non-null  float64
 4   countyfips    41742 non-null  object 
 5   State         29812 non-null  object 
 6   County        29812 non-null  object 
 7   County_Code   29812 non-null  object 
 8   Year          29812 non-null  float64
 9   Population    29812 non-null  Int64  
 10  State_Code    29812 non-null  object 
dtypes: Int64(2), float64(2), object(7)
memory usage: 3.6+ MB


In [21]:
# Drop redundant columns
final_shipment_data = shipment_with_population.drop(
    columns=["State", "County", "County_Code", "Year", "State_Code"]
)

# Check the first few rows of the resulting DataFrame
final_shipment_data.sample(10)

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,countyfips,Population
40608,WI,WAUSHARA,2006,3.325234e+06,55137,24690
18675,MN,TRAVERSE,2018,5.312418e+05,27155,<NA>
23468,NC,WASHINGTON,2012,3.446770e+06,37187,12662
12545,KS,MITCHELL,2010,1.298754e+06,20123,6336
27914,OH,MERCER,2015,5.545935e+06,39107,40735
18495,MN,RICE,2006,6.300715e+06,27131,61899
11656,IN,WASHINGTON,2017,9.793635e+06,18175,<NA>
4487,FL,MADISON,2011,4.646216e+06,12079,19134
24574,NE,FRANKLIN,2008,4.526787e+05,31061,3167
34513,TX,ELLIS,2018,3.068500e+07,48139,<NA>


In [22]:
# checking for null values before 2015
final_shipment_data[final_shipment_data["YEAR"] <= 2015].isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
MME             0
countyfips      0
Population      0
dtype: int64

In [23]:
# Checking for 0 Population values before 2015
final_shipment_data[final_shipment_data["YEAR"] <= 2015][
    final_shipment_data["Population"] == 0
]

/var/folders/cj/jkp7xyld4_q3x3rk7xdx7jwh0000gn/T/ipykernel_7836/285564038.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final_shipment_data[final_shipment_data["YEAR"] <= 2015][


,BUYER_STATE,BUYER_COUNTY,YEAR,MME,countyfips,Population


# Subsetting for Data 2015 and before

In [24]:
final_shipment_data = final_shipment_data[final_shipment_data["YEAR"] <= 2015]

In [25]:
# Calculate the MME per capita
final_shipment_data["MME_Per_Capita"] = (
    final_shipment_data["MME"] / final_shipment_data["Population"]
)

In [26]:
final_shipment_data.sample(5)

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,countyfips,Population,MME_Per_Capita
1727,AR,POPE,2015,2.954037e+07,05115,63642,464.164678
39382,WA,KITTITAS,2015,1.329391e+07,53037,43120,308.300229
37924,VA,HANOVER,2009,3.082829e+07,51085,99651,309.362537
18293,MN,MURRAY,2014,6.751052e+05,27101,8437,80.017204
19441,MO,HOWELL,2014,3.422605e+07,29091,40122,853.049548


In [27]:
final_shipment_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29812 entries, 0 to 41737
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   BUYER_STATE     29812 non-null  object 
 1   BUYER_COUNTY    29812 non-null  object 
 2   YEAR            29812 non-null  Int64  
 3   MME             29812 non-null  float64
 4   countyfips      29812 non-null  object 
 5   Population      29812 non-null  Int64  
 6   MME_Per_Capita  29812 non-null  Float64
dtypes: Float64(1), Int64(2), float64(1), object(3)
memory usage: 1.9+ MB


In [28]:
final_shipment_data.isna().sum()

BUYER_STATE       0
BUYER_COUNTY      0
YEAR              0
MME               0
countyfips        0
Population        0
MME_Per_Capita    0
dtype: int64

In [29]:
final_shipment_data.describe()

,YEAR,MME,Population,MME_Per_Capita
count,29812.0,2.981200e+04,29812.0,29812.0
mean,2010.500268,3.625713e+07,103607.488092,339.958515
std,2.872341,1.033919e+08,323038.580894,241.845681
min,2006.0,1.816200e+01,649.0,0.013443
25%,2008.0,2.738666e+06,12846.25,179.214235
50%,2011.0,9.043503e+06,27842.5,287.046357
75%,2013.0,2.782185e+07,71068.0,437.886585
max,2015.0,2.995369e+09,10077263.0,3156.932459


# Ignoring the Steps below since no Duplicate or 0 values are present further

In [ ]:
# Filter the DataFrame for years up to 2015
shipment_data_up_to_2015 = final_shipment_data[final_shipment_data["YEAR"] <= 2015]

# Filter rows in the filtered DataFrame where Population is NaN or zero
nan_or_zero_population_up_to_2015 = shipment_data_up_to_2015[
    (shipment_data_up_to_2015["Population"].isna())
    | (shipment_data_up_to_2015["Population"] == 0)
]

# Drop duplicates based on state, county, and year
unique_nan_or_zero_population_up_to_2015 = (
    nan_or_zero_population_up_to_2015.drop_duplicates(
        subset=["BUYER_STATE", "BUYER_COUNTY", "YEAR"]
    )
)

# Print the number of unique rows by year
print(unique_nan_or_zero_population_up_to_2015["YEAR"].value_counts())

In [ ]:
# Select state and county columns
unique_states_counties_with_nan_population = unique_nan_or_zero_population_up_to_2015[
    ["BUYER_STATE", "BUYER_COUNTY"]
]

# Drop duplicates to get unique combinations
unique_state_county_combinations = (
    unique_states_counties_with_nan_population.drop_duplicates()
)

In [ ]:
# Get unique state names for NaN population values
unique_states_with_nan_population = unique_nan_or_zero_population_up_to_2015[
    "BUYER_STATE"
].unique()

# Print the unique state names
print(unique_states_with_nan_population)

In [ ]:
# Extract the unique states with NaN or zero population
states_to_drop = unique_nan_or_zero_population_up_to_2015["BUYER_STATE"].unique()

# Filter out data from these states in the 'shipment_data_up_to_2015' DataFrame
filtered_shipment_data = shipment_data_up_to_2015[
    ~shipment_data_up_to_2015["BUYER_STATE"].isin(states_to_drop)
]

# Check the resulting DataFrame
print(filtered_shipment_data.sample(5))

In [ ]:
# Check for NaN values in all columns of the filtered dataset
nan_columns_exist = filtered_shipment_data.isna().any()

# Print the result
print(nan_columns_exist)

In [ ]:
# Define the file path and name for the parquet file
file_path = "../../Data/processed/shipment_corrected.parquet"

# Save the DataFrame to a parquet file
# filtered_shipment_data.to_parquet(file_path)